In [1]:
import sys, json, itertools
from pathlib import Path
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split,TensorDataset
import numpy as np
import pandas as pd
from transformers import T5Tokenizer, T5Config, T5ForConditionalGeneration, T5Model
import pytorch_lightning as pl
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [13]:
with open("../data/ingredients.json", 'r') as f:
    ingredients = dict(json.load(f))

In [2]:
df=pd.read_csv("seq2seq_4335716.csv")
df

,input_seq,output_seq
0,brown meat in a big skillet .,USE TSKILLET STOVE_MED PUT INJApA96N STOVE_MED...
1,"remove from heat and drain the fat , if any .",MOVE_CONTENTS STOVE_MED FAUCET_OFF USE TSKILLE...
2,"add the onion , celery and tomaotes .",MOVE_CONTENTS STOVE_MED COUNTER1 MOVE_CONTENTS...
3,stir until well mixed .,CHEF_CHECK LTEXTURE COUNTER1
4,place a layer of shredded cabbage in the botto...,USE TBAKE_DISH COUNTER2 USE TCOOK_SPRAY COUNTE...
...,...,...
496,add the cabbage and cook the vegetable mixture...,PUT IsxiK2rPw STOVE_MED CHEF_CHECK LTIME STOVE...
497,"then cover with hot water , season with walt a...",USE TFOIL STOVE_MED PUT IzrIHcuDJ STOVE_MED PU...
498,add the rice and cook for 15 minutes longer .,PUT IN2e0UIJI STOVE_MED CHEF_CHECK LTIME STOVE...
499,"add the cheese , stir gently and turn off the ...",PUT I_qDx9v7e STOVE_MED CHEF_CHECK LTIME STOVE...


In [3]:
sys.path.append('/home/ubuntu/pycharm/recipe_parsing/src')
from recipe_prep import *
from recipe_t5 import *

In [4]:
vocab, token_types = t5_extra_vocab(31000)
idx2vocab={v:k for k,v in vocab.items()}
# model = T5Recipe.from_pretrained('/home/ubuntu/recipe_model')
model = T5Recipe.from_pretrained('/home/ubuntu/trained_models/recipe_large_10')
# model = T5ForConditionalGeneration.from_pretrained('/home/ubuntu/recipe_model')

In [5]:
txt = "cook rice until tender"
input_ids = tokenizer.encode(txt, return_tensors='pt')
input_ids

tensor([[3989, 6605,  552, 9024,    1]])

In [6]:
triplet_types = (token_types['commands'],
token_types['args']+token_types['resources'],
token_types['resources'])

In [7]:
min_length=10
type_constraints = []
# type_constraints = list(itertools.chain(*itertools.repeat(triplet_types,min_length)))
model.set_generation_constraints(type_constraints)
output_ids = model.generate(input_ids=input_ids, min_length=min_length)
output_ids

BOOM


[31002,
 31149,
 31013,
 31000,
 31125,
 31013,
 31000,
 31125,
 31013,
 31000,
 31125,
 31013,
 31000,
 31125,
 31013,
 31000,
 31125,
 31013,
 31000]

In [28]:
def code_as_dataframe(output_ids):
    code = [ingredients.get(idx2vocab[i],idx2vocab[i]) for i in output_ids]
    df_code = []
    curr=[]
    for i,c in enumerate(code):
        if i%3==0:
            df_code.append(curr)
            curr=[c]
        else:
            curr.append(c)
    if len(curr)==3:
        df_code.append(curr)
    df_code = pd.DataFrame(df_code, columns = ["cmd", "arg", "res"]).dropna(how='all')
    return df_code
code_as_dataframe(output_ids)

,cmd,arg,res
1,USE,Sauce Pan,STOVE_MED
2,PUT,onion,STOVE_MED
3,PUT,onion,STOVE_MED
4,PUT,onion,STOVE_MED
5,PUT,onion,STOVE_MED
6,PUT,onion,STOVE_MED


In [32]:
def run_model_on_text(txt, min_length=10):
    input_ids = tokenizer.encode(txt, return_tensors='pt')
    output_ids = model.generate(input_ids=input_ids, min_length=min_length)
    return code_as_dataframe(output_ids)

run_model_on_text("freeze water until dry, cook for 10 minutes, let sit until cool")

BOOM


,cmd,arg,res
1,USE,Sauce Pan,STOVE_MED
2,PUT,onion,STOVE_MED
3,PUT,onion,STOVE_MED
4,PUT,onion,STOVE_MED
5,PUT,onion,STOVE_MED
6,PUT,onion,STOVE_MED
